License: BSD-3-Clause

Copyright the MNE-Python contributors.

Edited by: Edouard Delaire, Ilian Azz

# Install PIP dependencies

In [2]:
# Able to export maps as movies (optional)
%pip install imageio
# to convert the notebook to html, pdf, etc. (optional)
%pip install nbconvert
%pip install scipy
%pip install mne==1.7.0
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install darkdetect
# HERE 
# !!!!! The matlabengine version should match the version of MATLAB installed on your system !!!!!
# Check : https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html#
#%pip install matlabengine==23.2.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Load MNE model

In [1]:
import mne

from mne.datasets import sample

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
fwd_fname = meg_path / "sample_audvis-meg-eeg-oct-6-fwd.fif"
ave_fname = meg_path / "sample_audvis-ave.fif"
cov_fname = meg_path / "sample_audvis-shrunk-cov.fif"
subjects_dir = data_path / "subjects"
condition = "Left Auditory"

# Read noise covariance matrix
noise_cov = mne.read_cov(cov_fname)
# Handling average file
evoked = mne.read_evokeds(ave_fname, condition=condition, baseline=(None, 0))
evoked.crop(tmin=0.04, tmax=0.18)

evoked = evoked.pick(picks="meg", exclude="bads")
# Handling forward solution
forward = mne.read_forward_solution(fwd_fname)

# Convertir à un modèle à orientation libre
# force_fixed: Orientation fixe ou libre
# surf_ori: orientaton vers la surface
forward = mne.convert_forward_solution(forward, force_fixed=True, surf_ori=True)

    365 x 365 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 59) active
Reading /Users/edelaire1/mne_data/MNE-sample-data/MEG/sample/sample_audvis-ave.fif ...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Left Auditory)
        0 CTF compensation matrices available
        nave = 55 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
Reading forward solution from /Users/edelaire1/mne_data/MNE-sample-data/MEG/sample/sample_audvis-meg-eeg-oct-6-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added..

# Start & Configure matlab engine
## Start engine and add path

In [4]:
# Start MATLAB engine
import bestpython

matlab_wrapper = bestpython.MatlabWrapper("/Users/edelaire1/.brainstorm/plugins/brainentropy/best-brainstorm-master")

## Define MEM options

In [5]:
mem_options = bestpython.MEMOptions(
    data_modality="MEG",
    time_segment = [0.04, 0.18],
    baseline_time = [0.04, 0.1]
)

# Call solver

In [ ]:
stc = matlab_wrapper.mem_solver(evoked, forward, noise_cov, MEMOptions=mem_options)

info["bads"] and noise_cov["bads"] do not match, excluding bad channels from both
Computing inverse operator with 305 channels.
    305 out of 366 channels remain after picking
Selected 305 channels
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 3)
Computing rank from covariance with rank=None
    Using tolerance 3.5e-13 (2.2e-16 eps * 305 dim * 5.2  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Setting small MEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
-- number of adjacent vertices : 8196


/Users/edelaire1/Documents/software/best-python/bestpython/solver.py:105: RuntimeWarning: 8.5% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  vertex_connectivity_matrix = mne.spatial_src_adjacency(forward['src'])




===== pipeline cMEM
BLAS: trying environment BLAS_VERSION...
BLAS: loading libmwAF_BLAS_ilp64.dylib
BLAS: loaded libmwAF_BLAS_ilp64.dylib@0x2f02df140
cMEM, be_main_source ... done.
cMEM, be_main_leadfields ... done.
cMEM, solving MNE by L-curve ...done. 
cMEM, stable clustering ...
cMEM, stable clustering: dimension of the signal subspace 3, for inertia > 0.95
MEM : New noise covariance is computed using baseline
cMEM, No multimodalities ... 
 done.
cMEM, MEM at each samples (85 samples, may be done in parallel):
Multiresolution sample (j,t): j=0 corresponds to the sampling scale.
Sample   1( 0,0.042):
		264 clusters,
		1551 iter.
		Entropy drop:-631.2
Sample   2( 0,0.043):
		264 clusters,
		1533 iter.
		Entropy drop:-360.0


# Show results

## Brain Map of the solutions

In [ ]:
initial_time = -0.1
brain_cmem = stc_cmem.plot(
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    # clim=dict(kind="value", lims=[0, 5e-11, 1e-10]),
    smoothing_steps=7,
    hemi='both',
)

brain_cmem.add_text(0.1, 0.9, "cMEM", "title", font_size=14)

RuntimeError: Could not load any valid 3D backend
pyvistaqt: No Qt bindings could be found
notebook: No module named 'pyvista'

 install pyvistaqt, using pip or conda:
'pip install pyvistaqt'
'conda install -c conda-forge pyvistaqt'

 or install ipywidgets, if using a notebook backend
'pip install ipywidgets'
'conda install -c conda-forge ipywidgets'

## Amplitude for each source and vetex

## Compare with MNE Default method

In [ ]:
from mne import read_source_estimate
from mne.minimum_norm import apply_inverse, make_inverse_operator

fname_stc = meg_path / "sample_audvis-meg"

inverse_operator = make_inverse_operator(
    evoked.info, forward, noise_cov, loose=0, depth=0
)

stc_mne = apply_inverse(
    evoked,
    inverse_operator,
    pick_ori=None,
    method="MNE"
)

# Define plotting parameters
surfer_kwargs = dict(
    hemi="both",
    subjects_dir=subjects_dir,
    smoothing_steps=7,
)

# Plot surface
brain_mne = stc_mne.plot(**surfer_kwargs)
brain_mne.add_text(0.1, 0.9, "MNE", "title", font_size=14)

## Show maps with absolute values

In [ ]:
stc_cmem_abs = stc_cmem.copy()
stc_cmem_abs.data = np.abs(stc_cmem_abs.data)

brain_cmem_abs = stc_cmem_abs.plot(
    subject="sample",
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    smoothing_steps=7,
    hemi='both',
    title="cMEM abs",
)

brain_cmem_abs.add_text(0.1, 0.9, "cMEM abs", "title", font_size=14)

stc_mne_abs = stc_mne.copy()
stc_mne_abs.data = np.abs(stc_mne_abs.data)

# Plot the result
brain_mne_abs = stc_mne_abs.plot(
    subject="sample",
    subjects_dir=subjects_dir,
    initial_time=initial_time,
    smoothing_steps=7,
    hemi='both',
)

brain_mne_abs.add_text(0.1, 0.9, "MNE abs", "title", font_size=14)

Using control points [3.78270031e-14 7.02257232e-14 7.92968607e-13]


Using control points [1.69554860e-10 2.07091805e-10 6.66194162e-10]


Using control points [4.93664253e-13 5.96855178e-13 1.74351802e-12]
